In [ ]:
# Method 1: Using numpy
import numpy as np
a = np.array(x)
b = np.array(y)
dist = np.linalg.norm(a-b)

# Method 2: Using scipy
from scipy.spatial import distance
dst = distance.euclidean(a,b)

# numpy is faster than scipy

In [118]:
import pandas as pd
import numpy as np
data = pd.read_excel('HealthViz County Dataset 6.19.17.xlsx',skiprows=0, header=1, index_col=0)
data.index.name=None
data.drop(data.columns[[0,1]], axis=1, inplace=True)

In [121]:
# Step 1: Extract the target row out (assuming the targe place has the GEOID)
# Step 2: Get the Euclidean distance betweem each place and the target one
# Step 3: Sort the dataframe based on the arising order of the distance
# Step 4: Get the list of top N similar places, maybe all the values for comparison

def drop_null(df):
    '''
    Drop the columns and rows with any null value
    '''
    
    df.dropna(axis=1, how='any', inplace=True)
    df.dropna(axis=0, how='any', inplace=True)
    return df

def drop_the_targe_row(df, GEOID):
    '''
    Drop the target row from the orginal dataframe
        GEOID: index, int
    '''
    df = df.drop([GEOID])
    return df

def get_distance(GEOID_target, GEOID):
    '''
    Get the Euclidean distance between two sets of values
        GEOID_target, GEOID: index, int
    '''
    a = data.loc[[GEOID_target]].values # df.values
    b = data.loc[[GEOID]].values
    dist = np.linalg.norm(a-b)
    return dist

def  get_top_n_similar(data, GEOID_target, n):
    '''
    Get the N places which are similar to the target place
        data: dataframe
        GEOID_target: index, int
        n: int
    '''
    df = drop_null(data)
    df = drop_the_targe_row(df, GEOID_target)
    df['distance'] = df.apply(lambda x: get_distance(GEOID_target, x.name), axis=1)
    df = df.sort_values('distance', ascending=True)
    index_list = df.head(n).index.tolist()
    return index_list
    

In [122]:
get_top_n_similar(data, 1003, 6)

[22103, 45091, 30111, 53057, 17119, 37097]